In [31]:
import boto3
import os
import json
import csv
import pandas as pd 
import xlsxwriter

In [32]:
def get_home_dir():
    """Get home directory of the user based on OS."""

    if os.name == "nt":
        return os.path.expanduser("~\\.aws\\") # Windows
    else:
        return os.path.expanduser("~/.aws/") # Linux and MAC

In [33]:
def get_aws_credentials():
    """Read AWS credentials from session.json file."""
    home_dir = get_home_dir()
    full_path = os.path.join(home_dir, "session_token.json")

    with open(full_path) as f:
        credentials = json.load(f)["Credentials"]

    return credentials

In [34]:
# Read the AWS credentials from a local session.json file in home/user/.aws/ directory
credentials = get_aws_credentials()

In [35]:
# Connect to AWS Client
rds_client = boto3.client(
    "rds",
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretAccessKey"],
    aws_session_token=credentials["SessionToken"]
)

ec2_client = boto3.client(
    "ec2",
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretAccessKey"],
    aws_session_token=credentials["SessionToken"]
)

In [36]:
vpc_list = []
with open("vpc_ids.txt", "r") as f:
    vpc_list = [line.strip() for line in f.readlines()]

In [37]:
vpc_list

['vpc-b50046d0',
 'vpc-d50f49b0',
 'vpc-959d26f1',
 'vpc-840147e1',
 'vpc-84a444e0',
 'vpc-72df7917',
 'vpc-6c7f9409',
 'vpc-7a7f941f',
 'vpc-c52bc0a0',
 'vpc-0978b06c']

In [38]:
describe_rds_response = rds_client.describe_db_instances()

In [39]:
describe_rds_response

{'DBInstances': [{'DBInstanceIdentifier': 'appeals-preprod-caseflow-aurora-primary',
   'DBInstanceClass': 'db.t3.medium',
   'Engine': 'aurora-postgresql',
   'DBInstanceStatus': 'available',
   'MasterUsername': 'caseflow',
   'DBName': 'caseflow_certification',
   'Endpoint': {'Address': 'appeals-preprod-caseflow-aurora-primary.cdqbofmbcmtd.us-gov-west-1.rds.amazonaws.com',
    'Port': 5432,
    'HostedZoneId': 'Z2YROVNTF0QJSQ'},
   'AllocatedStorage': 20,
   'InstanceCreateTime': datetime.datetime(2021, 8, 20, 18, 46, 40, 813000, tzinfo=tzutc()),
   'PreferredBackupWindow': '08:47-09:17',
   'BackupRetentionPeriod': 7,
   'DBSecurityGroups': [],
   'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-04a768d794318de67',
     'Status': 'active'}],
   'DBParameterGroups': [{'DBParameterGroupName': 'default.aurora-postgresql11',
     'ParameterApplyStatus': 'in-sync'}],
   'AvailabilityZone': 'us-gov-west-1b',
   'DBSubnetGroup': {'DBSubnetGroupName': 'default-vpc-ef7cfa8a',
    'DBSubnet

In [40]:
get_db_instances = describe_rds_response.get("DBInstances")

In [41]:
get_db_instances

[{'DBInstanceIdentifier': 'appeals-preprod-caseflow-aurora-primary',
  'DBInstanceClass': 'db.t3.medium',
  'Engine': 'aurora-postgresql',
  'DBInstanceStatus': 'available',
  'MasterUsername': 'caseflow',
  'DBName': 'caseflow_certification',
  'Endpoint': {'Address': 'appeals-preprod-caseflow-aurora-primary.cdqbofmbcmtd.us-gov-west-1.rds.amazonaws.com',
   'Port': 5432,
   'HostedZoneId': 'Z2YROVNTF0QJSQ'},
  'AllocatedStorage': 20,
  'InstanceCreateTime': datetime.datetime(2021, 8, 20, 18, 46, 40, 813000, tzinfo=tzutc()),
  'PreferredBackupWindow': '08:47-09:17',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-04a768d794318de67',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.aurora-postgresql11',
    'ParameterApplyStatus': 'in-sync'}],
  'AvailabilityZone': 'us-gov-west-1b',
  'DBSubnetGroup': {'DBSubnetGroupName': 'default-vpc-ef7cfa8a',
   'DBSubnetGroupDescription': 'Database subnet 

In [42]:
len(get_db_instances)

47

In [43]:
get_only_the_vpc_in_vpc_list = []

for item in get_db_instances:
    check_vpc_id = item.get("DBSubnetGroup").get("VpcId")
    for row in vpc_list:
        if row == check_vpc_id:
            get_only_the_vpc_in_vpc_list.append(item)
        else:
            pass

In [44]:
len(get_only_the_vpc_in_vpc_list)

26

In [45]:
get_only_the_vpc_in_vpc_list

[{'DBInstanceIdentifier': 'dsva-cms-dev-db',
  'DBInstanceClass': 'db.m5.4xlarge',
  'Engine': 'mariadb',
  'DBInstanceStatus': 'available',
  'MasterUsername': 'master',
  'DBName': 'dsva_cms_dev',
  'Endpoint': {'Address': 'dsva-cms-dev-db.cdqbofmbcmtd.us-gov-west-1.rds.amazonaws.com',
   'Port': 3306,
   'HostedZoneId': 'Z2YROVNTF0QJSQ'},
  'AllocatedStorage': 100,
  'InstanceCreateTime': datetime.datetime(2021, 2, 11, 1, 56, 46, 951000, tzinfo=tzutc()),
  'PreferredBackupWindow': '07:24-07:54',
  'BackupRetentionPeriod': 14,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-4b466d2c',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'cms-dev-mariadb-10-5',
    'ParameterApplyStatus': 'in-sync'}],
  'AvailabilityZone': 'us-gov-west-1a',
  'DBSubnetGroup': {'DBSubnetGroupName': 'dsva-vagov-dev-cms-rds-sng',
   'DBSubnetGroupDescription': 'RDS subnet group for cms',
   'VpcId': 'vpc-b50046d0',
   'SubnetGroupStatus': 'Complete',
   

In [46]:
get_rds_info = []

for item in get_only_the_vpc_in_vpc_list:
    db_instance_identifier = item.get("DBInstanceIdentifier")
    engine = item.get("Engine")
    db_name = item.get("DBName")
    port = item.get("Endpoint").get("Port")
    vpc_id = item.get("DBSubnetGroup").get("VpcId")
    rds_sg = item.get("VpcSecurityGroups")

    rds_sg_id = []

    for row in rds_sg:
        rds_sg_id.append(row.get("VpcSecurityGroupId"))

    get_rds_info.append([db_instance_identifier, engine, db_name, port, vpc_id, rds_sg_id])

In [47]:
len(get_rds_info)

26

In [48]:
get_rds_info

[['dsva-cms-dev-db',
  'mariadb',
  'dsva_cms_dev',
  3306,
  'vpc-b50046d0',
  ['sg-4b466d2c']],
 ['dsva-cms-prod-db',
  'mariadb',
  'dsva_cms_prod',
  3306,
  'vpc-d50f49b0',
  ['sg-b38bb3d4']],
 ['dsva-cms-staging-db',
  'mariadb',
  'dsva_cms_staging',
  3306,
  'vpc-840147e1',
  ['sg-d690a8b1']],
 ['dsva-cms-test-dev-db',
  'mariadb',
  'dsva_cms_dev',
  3306,
  'vpc-b50046d0',
  ['sg-e8df7b91']],
 ['dsva-cms-test-prod-db',
  'mariadb',
  'dsva_cms_prod',
  3306,
  'vpc-d50f49b0',
  ['sg-64db7f1d']],
 ['dsva-cms-test-staging-db',
  'mariadb',
  'dsva_cms_staging',
  3306,
  'vpc-840147e1',
  ['sg-aaa501d3']],
 ['dsva-gi-bill-data-service-dev',
  'postgres',
  'dsva_gids_dev',
  5432,
  'vpc-b50046d0',
  ['sg-bd2702db']],
 ['dsva-gi-bill-data-service-prod',
  'postgres',
  'dsva_gids_prod',
  5432,
  'vpc-d50f49b0',
  ['sg-df7c5db9']],
 ['dsva-gi-bill-data-service-staging',
  'postgres',
  'dsva_gids_staging',
  5432,
  'vpc-840147e1',
  ['sg-ba2b0edc']],
 ['dsva-lighthouse-ux-too

In [49]:
security_group_list = []

for item in get_rds_info:
    security_group_of_the_item = item[5]

    # append() adds one element to the list at a time
    # extend() method is used to append one list to another list
    # extend() adds multiple items to the list at a time
    security_group_list.extend(security_group_of_the_item)

In [50]:
security_group_list

['sg-4b466d2c',
 'sg-b38bb3d4',
 'sg-d690a8b1',
 'sg-e8df7b91',
 'sg-64db7f1d',
 'sg-aaa501d3',
 'sg-bd2702db',
 'sg-df7c5db9',
 'sg-ba2b0edc',
 'sg-04921927dc74e3cb6',
 'sg-03627397dd3ecb4de',
 'sg-1a06ef7d',
 'sg-00734f4482cb11ceb',
 'sg-00734f4482cb11ceb',
 'sg-0376c9d13bf948630',
 'sg-04f859724325f1f78',
 'sg-04f859724325f1f78',
 'sg-03ca43c3cab84a3c4',
 'sg-0953fde0772143261',
 'sg-09f3d24216bba51c9',
 'sg-0f447c3dfa1127aa3',
 'sg-7aaef61c',
 'sg-d33fe7b7',
 'sg-0aba386d',
 'sg-d37c5db5',
 'sg-4122d127',
 'sg-32aef654']

In [51]:
# remove_duplicate_rds_sg_id = []

# for item in security_group_list:
#     if item not in remove_duplicate_rds_sg_id:
#         remove_duplicate_rds_sg_id.append(item)

In [52]:
describe_security_groups_response = ec2_client.describe_security_groups(
    Filters=[
        {
            "Name": "group-id",
            "Values": security_group_list,
        }
    ]
)

In [53]:
get_security_groups = describe_security_groups_response.get("SecurityGroups")

In [54]:
get_security_groups

[{'Description': 'Security group that controls access to the Keycloak RDS Database',
  'GroupName': 'dsva-vagov-dev-keycloak-db-sg',
  'IpPermissions': [{'FromPort': 5432,
    'IpProtocol': 'tcp',
    'IpRanges': [],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'ToPort': 5432,
    'UserIdGroupPairs': [{'Description': 'Ingress to DB',
      'GroupId': 'sg-0a313736ce4b5605b',
      'UserId': '008577686731'}]}],
  'OwnerId': '008577686731',
  'GroupId': 'sg-00734f4482cb11ceb',
  'IpPermissionsEgress': [{'IpProtocol': '-1',
    'IpRanges': [{'CidrIp': '10.0.0.0/8'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'UserIdGroupPairs': []}],
  'VpcId': 'vpc-b50046d0'},
 {'Description': 'Security group that controls access to the platform-console-ui dev database',
  'GroupName': 'dsva-vagov-dev-platform-console-ui-db-sg',
  'IpPermissions': [{'FromPort': 5432,
    'IpProtocol': 'tcp',
    'IpRanges': [],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'ToPort': 5432,
    'UserIdGr

In [55]:
get_security_group_name_and_id = []

for item in get_security_groups:
    group_name = item.get("GroupName")
    group_id = item.get("GroupId")

    get_security_group_name_and_id.append([group_name, group_id])

In [56]:
get_security_group_name_and_id

[['dsva-vagov-dev-keycloak-db-sg', 'sg-00734f4482cb11ceb'],
 ['dsva-vagov-dev-platform-console-ui-db-sg', 'sg-03627397dd3ecb4de'],
 ['dsva-vagov-utility-keycloak-db-sg', 'sg-0376c9d13bf948630'],
 ['dsva-vagov-staging-platform-console-ui-db-sg', 'sg-03ca43c3cab84a3c4'],
 ['dsva-vagov-dev-lut-db-sg', 'sg-04921927dc74e3cb6'],
 ['dsva-vagov-sentry-rds-sg', 'sg-04f859724325f1f78'],
 ['dsva-vagov-utility-pact-broker', 'sg-0953fde0772143261'],
 ['dsva-vagov-utility-platform-console-api-db-sg', 'sg-09f3d24216bba51c9'],
 ['dsva-vagov-dev-vets-api-server-sg', 'sg-0aba386d'],
 ['dsva-vagov-utility-platform-console-ui-db-sg', 'sg-0f447c3dfa1127aa3'],
 ['dsva-vagov-utility-grafana-rds-sg', 'sg-1a06ef7d'],
 ['dsva-vagov-staging-vets-api-db-sg', 'sg-32aef654'],
 ['dsva-vagov-sandbox-vets-api-db-sg', 'sg-4122d127'],
 ['dsva-vagov-dev-cms-rds-sg', 'sg-4b466d2c'],
 ['dsva-vagov-prod-cms-test-rds-sg', 'sg-64db7f1d'],
 ['dsva-vagov-dev-vets-api-db-sg', 'sg-7aaef61c'],
 ['dsva-vagov-staging-cms-test-rds-sg

In [57]:
for item in get_rds_info:
    # security group ids of the item
    item_sg_ids = item[5]

    # assume security groud id not avialable
    security_group_id_found = False

    for sg_id in item_sg_ids:
        # for each security group id of the item, check if the id found
        # in get_security_group_name_and_id
        # if so, we add security group name besides the id

        for sg_info in get_security_group_name_and_id:
            # check if security group id matches 
            if sg_id == sg_info[1]:
                item.append(sg_info[0]) #add the security group name

                # initial assumption was wrong, we found the security group id
                security_group_id_found = True

                # no need to check the rest
                break

        # after looking through all the get_security_group_name_and_id
        # check if we found security group id or not 
        if security_group_id_found == False:
                item.append('security group id not found')


In [58]:
get_rds_info 

[['dsva-cms-dev-db',
  'mariadb',
  'dsva_cms_dev',
  3306,
  'vpc-b50046d0',
  ['sg-4b466d2c'],
  'dsva-vagov-dev-cms-rds-sg'],
 ['dsva-cms-prod-db',
  'mariadb',
  'dsva_cms_prod',
  3306,
  'vpc-d50f49b0',
  ['sg-b38bb3d4'],
  'dsva-vagov-prod-cms-rds-sg'],
 ['dsva-cms-staging-db',
  'mariadb',
  'dsva_cms_staging',
  3306,
  'vpc-840147e1',
  ['sg-d690a8b1'],
  'dsva-vagov-staging-cms-rds-sg'],
 ['dsva-cms-test-dev-db',
  'mariadb',
  'dsva_cms_dev',
  3306,
  'vpc-b50046d0',
  ['sg-e8df7b91'],
  'dsva-vagov-dev-cms-test-rds-sg'],
 ['dsva-cms-test-prod-db',
  'mariadb',
  'dsva_cms_prod',
  3306,
  'vpc-d50f49b0',
  ['sg-64db7f1d'],
  'dsva-vagov-prod-cms-test-rds-sg'],
 ['dsva-cms-test-staging-db',
  'mariadb',
  'dsva_cms_staging',
  3306,
  'vpc-840147e1',
  ['sg-aaa501d3'],
  'dsva-vagov-staging-cms-test-rds-sg'],
 ['dsva-gi-bill-data-service-dev',
  'postgres',
  'dsva_gids_dev',
  5432,
  'vpc-b50046d0',
  ['sg-bd2702db'],
  'dsva-vagov-dev-gids-db-sg'],
 ['dsva-gi-bill-data-

In [59]:
rds_security_group_list = []
with open("rds_security_group_list.txt", "r") as f:
    rds_security_group_list = [line.strip() for line in f.readlines()]

In [60]:
def highlight(value):
    """Highlight a cell based on value."""

    if "\t" in value:
        color = "yellow"
    else:
        return

    return f"background-color: {color}"


filename = "rds_scan_coverage.xlsx"

with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
    all_sheets = {}

    for db_info in get_rds_info:
        rds_instance_identifier = db_info[0]
        rds_engine = db_info[1]
        rds_db_name = db_info[2]
        rds_port = db_info[3]
        rds_vpc_id = db_info[4]
        rds_security_group_ids = db_info[5]

        # nubmber of security groups for the item
        num_of_sg_ids = len(rds_security_group_ids)

        # when the instance has more than one security group
        # each security group is added as a new row in the excel
        rows = {}
        rows[rds_vpc_id] = []

        # if we have one security group id, the last one element is security group name
        # if we have two, the last two elements are security group names
        # if we have n, the last n elements are security group names
        # where n is number of security group ids of the instance
        # in our case n = num_of_sg_ids

        # so db_info[-num_of_sg_ids:] means last n elements (security group names)
        # that are correspondent for each security group id

        for sg_id, sg_name in zip(rds_security_group_ids, db_info[-num_of_sg_ids:]):
            # check if the group name exists in our list of target groups
            if sg_name in rds_security_group_list:
                # mark it to be highlighted (cells with \t added are to be highlighted)
                sg_name = sg_name + "\t"

            # if rows.get(rds_vpc_id):
            #     # a row to be inserted in the excel with group info only
            #     row = ["", "", "", "", sg_id, sg_name]
            #     rows[rds_vpc_id].append(row)

            if rows[rds_vpc_id] != []:
                # a row to be inserted in the excel with group info only
                row = ["", "", "", "", sg_id, sg_name]
                rows[rds_vpc_id].append(row)

            else:
                # a row to be inserted in the excel with full instance info
                row = [
                    rds_instance_identifier,
                    rds_engine,
                    rds_db_name,
                    rds_port,
                    sg_id,
                    sg_name,
                ]
                rows[rds_vpc_id].append(row)

        # organize rows into different sheets
        if all_sheets.get(rds_vpc_id):
            # sheet is aleardy created
            all_sheets[rds_vpc_id].extend(rows[rds_vpc_id])
        else:
            # create a new sheet
            all_sheets[rds_vpc_id] = []
            all_sheets[rds_vpc_id].extend(rows[rds_vpc_id])

    for sheet in all_sheets:
        df = pd.DataFrame(
            all_sheets[sheet],
            columns=[
                "DBInstanceIdentifier",
                "Engine",
                "DBName",
                "Port",
                "SecurityGroups",
                "SecurityName",
            ],
        )

        # Apply style to highlight cells based on value
        df = df.style.applymap(highlight, subset=["SecurityName"])

        df.to_excel(writer, sheet_name=sheet, index=False)